### SparklyRGT Template: Baseline and Acquisition Analysis 

**Requirements**
* The data must be an excel file from MEDPC2XL (trial by trial data) 
* The data, sparklyRGT.py file, and this notebook must all be in the same folder

**Getting started: Please make a copy of this (sparklyRGT_template_2) for each analysis**
- Refer to sparklyRGT_documentation for function information
- Note: depending on your analysis, you will only have to complete certain sections of the sparklyRGT_documentation
- Note: feel free to create a personal template once you've become comfortable - this is just an example

In [2]:
import os
os.chdir('..')
import sparklyRGT as rgt 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import scipy.stats as stats
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows',100)

I am being executed!


***

# 1) Load data into Python



In [3]:
#remove the leading 'M' from the TF file subject numbers and convert to integer/float 

In [4]:
#add header names from BH03 to other files so they can be concatenated properly 

In [5]:
from os import listdir
#data needs to be loaded in from OSF
#either download them from OSF and upload to github, or load in directly from OSF 

#OSF files are CSVs (except BH03 is xlsx) and load_multiple_data loads in excel files
#sparklyrgt.py needs to be edited so that either excel files or CSVs can be loaded in

path = '../sparklyRGT_tutorial/data/'
file_names = [f for f in listdir(path)]

df = rgt.load_multiple_data(file_names, path, reset_sessions = False)

df.head()
df.to_csv('../sparklyRGT_tutorial/output.csv')

In [6]:

# set reset_sessions = false
# run all exclusions
# add:
# check if there's any rats that have less than 20 session numbers -- exclude them

# figure out which file is printing -- order of file 
# then run reset sections --> set session number 1 -> n for each subject number
# rats will have at least 20 sessions (after exclusion above)
# write function where it checks whether each subject has 1-20 session number

# save a csv of session 18-20 (after reset), 2 files, cue and classic


def check_sessions(df): ##checks that the 'Session' column has correct, and non-missing session numbers
    pd.set_option('display.max_rows', None) ##otherwise it will ... the middle rows (only give the head and tail)
    print(df.groupby(['Subject','StartDate','Session'])['Trial'].max())
    pd.set_option('display.max_rows',df.Subject.max()) ##this sets the number of displayed rows to the number of subjects

    

In [7]:
from os import listdir
path = '../sparklyRGT_tutorial/'
file_names = ['output.csv']

# df = rgt.load_data(file_names, reset_sessions = False)
df = pd.read_csv('output.csv')
# check_sessions(df)
# df = df.drop(df.columns[[0]], axis=1)

task_list = df.groupby(['MSN'])['Subject'].unique()

/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
subjects = df.drop_duplicates(subset=['Subject', 'Session'])
subjects_n = subjects[['Subject', 'Session']]
zero_session = []
for index, row in subjects_n.iterrows():
    if row["Session"] == 0:
        zero_session.append(row["Subject"])
print(zero_session)       
# drop subjects that had a 0 session
df = rgt.drop_subjects(df, zero_session)

[165.0, 166.0, 167.0, 168.0, 169.0, 170.0, 171.0, 172.0, 203.0, 209.0, 210.0, 211.0, 212.0, 213.0, 214.0, 215.0, 216.0, 204.0, 205.0, 206.0, 611.0, 615.0, 607.0, 608.0, 627.0, 631.0, 632.0, 623.0, 624.0, 617.0, 609.0, 610.0, 612.0, 614.0, 616.0, 701.0, 702.0, 703.0, 704.0, 705.0, 706.0, 707.0, 708.0, 717.0, 718.0, 719.0, 720.0, 721.0, 722.0, 723.0, 724.0, 709.0, 726.0, 710.0, 711.0, 712.0, 713.0, 714.0, 715.0, 716.0, 1009.0, 1010.0, 1011.0, 1012.0, 1013.0, 1014.0, 1015.0, 1016.0, 1125.0, 1126.0, 1127.0, 1129.0, 1130.0, 1131.0]


## Data cleaning

### Check session numbers for each rat and drop subjects

In [9]:
#missing data

#we need to only take rats that have consecutive sessions from 1 to 5
#some rats will be missing sessions and need to be excluded

# removes subjects where the first 5 sessions are not consecutive
subjects = df.drop_duplicates(subset=['Subject', 'Session'])
subjects_n = subjects[['Subject', 'Session']]
# sort sessions in ascending order to check for consecutive sessions
subjects_n = subjects_n.sort_values(by=['Subject', 'Session'])
# count number of subjects
n = subjects_n['Subject'].nunique()
drop_subs = []

i = 0
temp = subjects_n
# iterate through each subject and check if they have the first 5 sessions consecutive
while i < n:
    # look at first 5 rows from subject
    check_consec = temp.head()
    # get subject number
    num = check_consec['Subject'].iloc[0]
    # convert first 5 sessions into number list
    con_list = check_consec['Session'].tolist()
    # list -> int
    con_list = list(map(int, con_list))
    # check if list is not consecutive, if so, add to drop list
    if sorted(con_list) != list(range(min(con_list), max(con_list)+1)):
        drop_subs.append(num)
    i = i + 1
    # remove the subject from the list, move to next subject
    temp = temp[temp.Subject != num]
print(drop_subs)
# drop subjects that did not have consecutive sessions (5)
df2 = rgt.drop_subjects(df, drop_subs)
# df2

[425, 426, 427, 428, 925, 926, 927, 929, 930, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1118, 1318, 1322, 1326]


In [10]:
#check if there are any subjects that were run on more than one task version (including 5CSRT, FC)

#to display all the task names in the dataframe:
# df2.MSN.unique()
print(df2[df2['Subject']==1304].MSN.unique())

# drop duplicate rows
tasks = df2.drop_duplicates(subset=['Subject', 'MSN'])
# look at subjects and tasks
tasks_n = tasks[['Subject', 'MSN']]
# sort by subject number
tasks_n = tasks_n.sort_values(by=['Subject'])
# find subject numbers that appear more than once in list
tasks_n_dup = tasks_n[tasks_n.duplicated(['Subject'], keep=False)]
tasks_n_dup = tasks_n_dup.Subject.unique()
duplicate_tasks = []
# iterate through subjects with more than 1 task and save them to list
for i in tasks_n_dup:
    duplicate_tasks.append(i)
# print(duplicate_tasks)
#drop any subjects that were run on more than one task 
final_subjects = rgt.drop_subjects(df2, duplicate_tasks)

['rGT_A-cue-FC2' 'rGT_A-cue']


In [11]:
less_20 = final_subjects.drop_duplicates(subset=['Subject','Session'])
# look at subjects and tasks
less = less_20[['Subject', 'Session']]
less = less.sort_values(by=['Subject', 'Session'])
candidates = less['Subject'].unique()
rejects = []
for i in candidates:
    if ((less[less.Subject == i].shape[0]) < 20):
        rejects.append(i)
# print(rejects)
over_20_sessions = rgt.drop_subjects(final_subjects, rejects)
over_20_sessions.to_csv('over_20_sessions.csv')
print(over_20_sessions)

        Unnamed: 0        MSN StartDate StartTime  Subject  Group  Box  \
0                0  rGT_A-cue  01/23/16   8:13:19      173    0.0    1   
1                1  rGT_A-cue  01/23/16   8:13:19      173    0.0    1   
2                2  rGT_A-cue  01/23/16   8:13:19      173    0.0    1   
3                3  rGT_A-cue  01/23/16   8:13:19      173    0.0    1   
4                4  rGT_A-cue  01/23/16   8:13:19      173    0.0    1   
...            ...        ...       ...       ...      ...    ...  ...   
427433      721550  rGT_B-cue  11/15/18  16:10:57     1408    0.0    8   
427434      721551  rGT_B-cue  11/15/18  16:10:57     1408    0.0    8   
427435      721552  rGT_B-cue  11/15/18  16:10:57     1408    0.0    8   
427436      721553  rGT_B-cue  11/15/18  16:10:57     1408    0.0    8   
427437      721554  rGT_B-cue  11/15/18  16:10:57     1408    0.0    8   

       Experiment  Comment  Session  ...  Pun_Persev_H5  Pun_HeadEntry  \
0             0.0      NaN      1.0  

In [12]:
# reset sessions
# assuming order of data doesn't matter in ML algorithm
subs = over_20_sessions['Subject'].unique().tolist()
print(subs)

[173, 174, 175, 177, 178, 179, 180, 181, 182, 183, 184, 201, 202, 207, 208, 225, 226, 227, 228, 217, 218, 219, 220, 221, 222, 223, 224, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 325, 326, 327, 328, 329, 330, 331, 332, 317, 318, 319, 320, 321, 322, 323, 324, 402, 406, 401, 403, 407, 404, 408, 405, 417, 419, 423, 424, 420, 418, 421, 422, 430, 432, 431, 429, 409, 413, 410, 412, 415, 411, 414, 416, 509, 510, 511, 512, 513, 514, 515, 516, 525, 526, 527, 528, 529, 530, 531, 532, 517, 518, 519, 520, 521, 522, 523, 524, 613, 601, 602, 603, 604, 605, 606, 625, 626, 628, 629, 630, 618, 619, 620, 621, 622, 725, 727, 728, 729, 730, 731, 732, 928, 931, 932, 917, 918, 919, 920, 921, 922, 923, 924, 901, 902, 903, 904, 905, 906, 907, 908, 913, 914, 915, 916, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1128, 1132, 1117, 1119, 1120, 1121, 1122, 1123, 1124, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1409, 1410, 1411, 1412, 1413, 1

In [13]:
final_df = pd.read_csv('over_20_sessions.csv')
# check_sessions(final_df)

/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
for i in subs:
    temp = over_20_sessions.loc[over_20_sessions['Subject'] == i]
    # remove duplicates and sort by sessions
    no_duplicates = temp.drop_duplicates(subset=['Subject','Session'])
    sort_i = no_duplicates.sort_values(by=['StartDate'])
    # print(sort_i)
    new_session_number = 0
    for index, row in sort_i.iterrows():
        new_session_number = new_session_number + 1
        old_session_number = row['StartDate']
        final_df['Session'] = np.where(((final_df['Subject'] == i) & (final_df['StartDate'] == old_session_number)), new_session_number, final_df['Session'])
        #iterate through rows in temp and replace row that contain same values in the main df:over_20_sessions  

In [15]:
# check_sessions(final_df)

In [16]:
final_df.to_csv('sockeye_data.csv')

In [1]:
task_list = final_df.groupby(['MSN'])['Subject'].unique()
print(task_list)

NameError: name 'final_df' is not defined

In [39]:
len(np.concatenate(task_list[[task for task in final_df.MSN.unique() if 'Classic' in task]]))

78

In [43]:
# check_sessions(final_df)

In [ ]:
# split the dataframe by cued and classic rats, and save them as two separate CSVs (both with column headers)

#all cued tasks will have 'cue' in the MSN (A and B version)
#all classic tasks should have 'Classic' (A and B) - either rGT or RGT 

#upload to sparklyRGT/data 